In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

Launch_site = list(spacex_df['Launch Site'].unique())
Launch_site
options = [{'label': 'All Sites', 'value': 'ALL'}]
for record in Launch_site:
    options.append({'label': record, 'value': record})

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                             options = options, 
                                             value = 'ALL',
                                             placeholder="place holder here",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart', figure = {})),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0', 100: '100'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart',  figure = {})),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',
                      component_property='figure'),
              Input(component_id='site-dropdown', 
                     component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['class']==1]
        filtered_df = filtered_df[['Launch Site','class']]
        title = 'Pie chart for ' + entered_site + ' sites'
        fig = px.pie(filtered_df, names='Launch Site',  
        title=title)
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df[['Launch Site','class']]
        title = 'Pie chart for ' + entered_site + ' site'
        fig = px.pie(filtered_df, names='class', 
        title=title)
        return fig

 # TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',
                      component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, Payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'] >= Payload[0]]
    filtered_df = filtered_df[filtered_df['Payload Mass (kg)']<= Payload[1]]
    title = 'Correlation between Payload in the range [{} Kg - {} kg] and success for {}'.format(Payload[0], Payload[1], entered_site)
    if entered_site =='ALL':
        title = title + ' sites'
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", 
            color="Booster Version Category", title=title)
        return fig
    else:
        title = title + ' site'
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", 
            color="Booster Version Category", title=title)
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server() 